In [151]:
import pandas as pd
import numpy as np
import random
from scipy.spatial.distance import pdist, squareform
import itertools
from geopy.distance import geodesic
import os

In [152]:
# Leer datos
data = pd.read_excel('Candidatos_supermercados.xlsx')

In [153]:
# Obtener coordenadas y poblaciones
coords = data[['Latitud', 'Longitud']].values
populations = data['Poblacion500m'].values

In [154]:
# Calculo de las distancias en km
def calculate_geodesic_distances(coords):
    n = len(coords)
    geo_distances = np.zeros((n, n))

    for i in range(n):
        for j in range(i + 1, n):
            # Calcular la distancia entre dos puntos y asignarla tanto en [i, j] como en [j, i]
            distance_km = geodesic(coords[i], coords[j]).kilometers
            geo_distances[i, j] = distance_km
            geo_distances[j, i] = distance_km

    return geo_distances

In [155]:
len(coords)

60

In [156]:
# Calcular la matriz de distancias geodésicas en kilómetros
distances_km = calculate_geodesic_distances(coords)

In [157]:
# Arreglo de distancias entre puntos
distances_km

array([[0.        , 1.6935315 , 2.05349239, ..., 8.93488781, 8.88319365,
        8.45854008],
       [1.6935315 , 0.        , 0.68432877, ..., 8.51761045, 8.42557777,
        8.07197853],
       [2.05349239, 0.68432877, 0.        , ..., 7.85388592, 7.75785298,
        7.41282819],
       ...,
       [8.93488781, 8.51761045, 7.85388592, ..., 0.        , 0.22603489,
        0.48954373],
       [8.88319365, 8.42557777, 7.75785298, ..., 0.22603489, 0.        ,
        0.53575624],
       [8.45854008, 8.07197853, 7.41282819, ..., 0.48954373, 0.53575624,
        0.        ]])

In [158]:
distances_km.shape

(60, 60)

In [159]:
# Función de fitness con penalización
def fitness(individual, distances_km, populations):
  if all(distances_km[i, j] >= 1 for i in individual for j in individual if i < j):
    total_distance = np.sum([distances_km[i, j] for i in individual for j in individual if i < j])
    total_population = np.sum([populations[i] for i in individual])
  else :
    total_distance = 0
    total_population = 0
  return total_distance + total_population

In [160]:
# Función de fitness desagregado
def fitness_desagregado(individual, distances_km, populations):
  if all(distances_km[i, j] >= 1 for i in individual for j in individual if i < j):
    total_distance = np.sum([distances_km[i, j] for i in individual for j in individual if i < j])
    total_population = np.sum([populations[i] for i in individual])
  else :
    total_distance = 0
    total_population = 0
  return total_distance, total_population

In [161]:
# Función para generar un individuo
def generate_individual():
    while True:
        individual = random.sample(range(len(coords)), 10)
        if all(distances_km[i, j] >= 1 for i in individual for j in individual if i < j):
            return individual

In [162]:
# Inicializar población utilizando combinaciones válidas
population = [list(generate_individual()) for _ in range(100)]

In [163]:
population

[[21, 7, 29, 28, 49, 3, 0, 23, 25, 59],
 [28, 37, 7, 24, 29, 25, 0, 48, 43, 14],
 [28, 13, 26, 22, 42, 59, 51, 14, 17, 18],
 [31, 13, 28, 58, 26, 38, 0, 2, 14, 20],
 [30, 10, 1, 20, 37, 22, 59, 47, 53, 14],
 [59, 11, 25, 43, 29, 19, 39, 24, 27, 2],
 [27, 22, 10, 26, 39, 3, 32, 0, 2, 45],
 [2, 55, 3, 44, 27, 21, 48, 9, 25, 22],
 [15, 3, 12, 29, 47, 26, 27, 35, 54, 24],
 [28, 54, 19, 24, 45, 11, 10, 29, 38, 23],
 [55, 28, 47, 17, 32, 12, 23, 24, 21, 14],
 [25, 29, 50, 12, 17, 24, 23, 54, 33, 27],
 [25, 39, 29, 14, 24, 15, 19, 10, 56, 3],
 [31, 54, 57, 21, 5, 14, 25, 42, 2, 46],
 [29, 21, 34, 2, 15, 27, 12, 23, 53, 48],
 [21, 55, 14, 8, 48, 29, 2, 15, 3, 38],
 [47, 42, 26, 31, 3, 22, 1, 21, 54, 50],
 [3, 8, 27, 59, 18, 44, 23, 15, 51, 25],
 [30, 23, 51, 24, 21, 6, 18, 15, 41, 59],
 [42, 2, 28, 7, 58, 26, 29, 19, 37, 0],
 [15, 47, 26, 59, 23, 40, 20, 43, 14, 2],
 [59, 31, 41, 18, 54, 3, 12, 21, 45, 27],
 [0, 26, 59, 6, 2, 29, 14, 23, 48, 54],
 [1, 59, 0, 36, 26, 28, 21, 29, 54, 49],
 [51, 

In [164]:
len(population)

100

In [165]:
# Operador de selección: Torneo
def tournament_selection(population, fitness_values, k=3):
    selected = []
    for _ in range(len(population)):
        tournament = random.sample(list(zip(population, fitness_values)), k)
        winner = max(tournament, key=lambda x: x[1])
        selected.append(winner[0])
    return selected

In [166]:
# Operador de cruzamiento
def crossover(parent1, parent2):
    cut = random.randint(1, len(parent1) - 2)
    child1 = parent1[:cut] + [gene for gene in parent2[cut:] if gene not in parent1[:cut]]
    child2 = parent2[:cut] + [gene for gene in parent1[cut:] if gene not in parent2[:cut]]
    if len(child1) != 10 :
      child1 = parent1
    if len(child2) != 10 :
      child2 = parent2
    return [child1, child2]

In [167]:
# Operador de mutación
def mutate(individual, mutation_rate=0.1):
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            swap_with = random.randint(0, len(individual) - 1)
            individual[i], individual[swap_with] = individual[swap_with], individual[i]
    return individual

In [177]:
# Ciclo del algoritmo genético
population = [list(generate_individual()) for _ in range(100)]

for generation in range(500):
    # Calcular fitness
    fitness_values = [fitness(individual, distances_km, populations) for individual in population]

    # Selección
    selected_population = tournament_selection(population, fitness_values)

    # Cruzamiento y Mutación
    new_population = []
    for i in range(0, len(selected_population), 2):
        parent1, parent2 = selected_population[i], selected_population[i+1]
        children = crossover(parent1, parent2)
        new_population.extend([mutate(child) for child in children])

    # Reemplazar población antigua con la nueva
    population = new_population

# Mostrar resultados finales
fitness_values = [fitness(individual, distances_km, populations) for individual in population]
fitness_values_desagregado = [fitness_desagregado(individual, distances_km, populations) for individual in population]
best_individual = population[np.argmax(fitness_values)]
#print("Mejor individuo encontrado:", best_individual)
#print("Fitness del mejor individuo:", max(fitness_values))
#print("E1:", best_individual," ",max(fitness_values)," ",max(fitness_values_desagregado)[0]," ",max(fitness_values_desagregado)[1])

data = {"Individuo": [best_individual],
        "Fitness": [max(fitness_values)],
        "Sum_Distancia": [max(fitness_values_desagregado)[0]],
        "Sum_Población": [max(fitness_values_desagregado)[1]],
        }
df = pd.DataFrame(data)

pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 0)
pd.set_option('display.colheader_justify', 'left')
pd.set_option('display.expand_frame_repr', False)


filename = "resultados.csv"
if os.path.exists(filename):
  df = pd.read_csv(filename)
else:
  df = pd.DataFrame(columns=["Individuo", "Fitness", "Sum_Distancia", "Sum_Población"])

nuevo_resultado = pd.DataFrame(data)

df = pd.concat([df, nuevo_resultado], ignore_index=True)

df.to_csv(filename, index=False)



print(df)



  Individuo                                 Fitness        Sum_Distancia  Sum_Población
0  [14, 10, 19, 47, 54, 59, 15, 17, 23, 1]  173842.464108  242.464108     173600       
1     [48, 3, 9, 54, 0, 25, 14, 2, 21, 59]  170932.368526  232.368526     170700       
2   [3, 10, 50, 59, 18, 22, 21, 14, 54, 1]  175022.208551  222.208551     174800       
3     [0, 14, 24, 48, 2, 21, 54, 3, 4, 59]  171627.537591  227.537591     171400       
4  [10, 59, 19, 54, 50, 22, 24, 17, 14, 2]  173624.721159  224.721159     173400       
5  [15, 2, 21, 16, 55, 12, 14, 48, 22, 17]  173556.821550  206.821550     173350       
6   [19, 59, 3, 48, 14, 2, 13, 15, 22, 53]  174931.658704  231.658704     174700       
7   [59, 14, 3, 18, 54, 23, 1, 49, 10, 15]  175934.128645  234.128645     175700       
8  [17, 54, 14, 2, 46, 16, 59, 21, 12, 22]  172484.532434  234.532434     172250       
9   [22, 13, 14, 50, 3, 19, 59, 54, 15, 2]  175476.798474  226.798474     175250       


In [178]:
distances_km[48,14]

11.94496278705403

In [149]:
# Ordenar por la columna "fitness" en orden descendente
top_10_fitness = df.sort_values(by="Fitness", ascending=False).head(10)
print(top_10_fitness)

   Individuo                                  Fitness        Sum_Distancia  Sum_Población
5    [23, 1, 16, 14, 48, 21, 18, 15, 54, 59]  176433.585565  233.585565     176200       
8      [5, 49, 59, 2, 19, 14, 54, 23, 0, 24]  174827.903325  227.903325     174600       
7    [14, 59, 10, 15, 48, 18, 23, 24, 1, 53]  174790.537291  240.537291     174550       
9    [47, 12, 21, 14, 22, 2, 59, 50, 54, 16]  174653.788245  253.788245     174400       
3   [15, 18, 48, 14, 22, 54, 59, 12, 24, 21]  174526.597490  226.597490     174300       
4      [59, 14, 23, 2, 54, 24, 49, 21, 0, 9]  174330.061697  230.061697     174100       
2     [50, 54, 59, 24, 46, 15, 14, 22, 6, 2]  174052.129347  252.129347     173800       
0    [59, 2, 15, 14, 50, 23, 10, 54, 21, 24]  173932.673477  232.673477     173700       
6    [15, 10, 22, 17, 54, 21, 24, 59, 1, 14]  172854.747937  204.747937     172650       
10     [23, 59, 6, 0, 24, 19, 47, 53, 2, 14]  170938.073158  238.073158     170700       


In [179]:
# Borrar las filas de la data si queremos analizar desde vacio
df_vacio = pd.DataFrame(columns=df.columns)
df_vacio.to_csv(filename, index=False)